In [1]:
import re

class CAZy_data:
    def __init__(self,filename1,filename2):
        self.data,self.acc,self.seq=[],[],[]
        with open(filename1,'r',encoding='utf-8') as inpt:
            for each in inpt:
                self.data.append(each.rstrip().split('$'))
                
        with open(filename2,'r',encoding='utf-8') as inpt1:
            for each1 in inpt1:
                if each1.startswith('>'):
                    self.acc.append(each1.rstrip())
                else:
                    self.seq.append(each1.rstrip())
            
                  
    def data_fetch(self,typ,position):
        typ_data=[]
        if typ=='all':
            typ_data=self.data
        else:
            for each in self.data:
                mult=each[position].split(' ')
                if len(mult)==1:#### In case typ = EC, Multi EC number and protein with no EC number are ignore.
                    if mult[0]==typ:
                        typ_data.append(each)
        return typ_data
    
    def EC_GH(self,ec_no,gh_fam):
        self.fasta=[]
        cazy_ec=self.data_fetch(ec_no,1)
        cazy_gh=self.data_fetch(gh_fam,-1)
        self.common_data=[[i[0],i[1],i[3],i[4],i[-2],i[-1]] for i in cazy_ec if i in cazy_gh]
        rm_prt, rm_prt_fasta=[],[]
        for each in range(len(self.common_data)):
            t=self.common_data[each]
            if self.prtn_filter(t[0]):
                all_acc=t[3].split(' ')
                if all_acc[0]!='':
                    for e_acc in all_acc:
                        e_seq=self.seq_fetch(e_acc)
                        try:
                            create_error=0/len(e_seq) # to remove accession number which doesnt have hits
                            self.fasta.append(f'>{e_acc}${t[0]}${t[1]}${t[2]}${t[-2]}${t[-1]}')
                            self.fasta.append(e_seq[0])
                        except ZeroDivisionError:
                            rm_prt_fasta.append(e_acc)
            else:
                rm_prt.append(t)
#         print('Total number of sequences:',len(self.fasta)/2)
#         print('Number of removed partial or fragment proteins (CAZy):',len(rm_prt))
#         print('Number of removed partial or fragment proteins (Fasta):',len(rm_prt_fasta))
        return self.fasta,rm_prt,rm_prt_fasta
    def prtn_filter(self,prt_name):
        hit=1
        if re.search('partial|fragment',prt_name.lower()):
            hit-=1
        return hit
            
    def seq_fetch(self,accession):
        hits=[]
        temp=0
        for each in range(len(self.acc)):
            if re.search(f'{accession}\D',self.acc[each]):
                temp+=1
                
                if self.prtn_filter(self.acc[each]): # remove partial| fragment accession numbers from GenBank description
                    hits.append(self.seq[each])
        if temp>1:
            print(f'Multiple hits for {accession}')
        elif temp==0:
            print(f'No hits for {accession}')
        return hits
     
In_data=CAZy_data('cazy_char_10_6_22.txt','char_gh_23_6_22.txt')


In [2]:
cazy_acc_seq,rm_cazy,rm_genbank=In_data.EC_GH('all','GH1') # write all to fetch all the EC number or all the GH family

No hits for NP_578085.1
No hits for NP_577802.1
No hits for NP_578937.1
No hits for NP_578171.1
No hits for NP_142340.1
No hits for NP_142473.1
No hits for CAC24042.1
No hits for NP_344331.1
No hits for YP_256448.1
No hits for NP_111204.1
No hits for NP_391805.1
No hits for ABP70047.1
No hits for YP_066184.1
No hits for NP_242789.1
No hits for ABR73190.1
No hits for NP_386997.1
No hits for BAB45117.1
No hits for NP_372713.1
No hits for NP_721380.1
No hits for NP_721491.1
No hits for NP_721937.1
No hits for NP_345092.1
No hits for NP_269656.1
No hits for NP_625353.1
No hits for NP_631601.1
No hits for AAZ54975.1
No hits for ADI56259.1
No hits for NP_936184.1
No hits for NP_197972.1
No hits for AEE33889.1
No hits for NP_176375.1
No hits for AAG21562.1
No hits for NP_187303.1
No hits for AAK25950.1
No hits for AAK64096.1
No hits for AAG12895.1
No hits for AAG12767.1
No hits for NP_175558.1
No hits for NP_198505.1
No hits for AEZ01595.1
No hits for S50756
No hits for AAA83309.1
No hits for

In [3]:
cazy_acc_seq

['>ADL19795.1$multifunctional β-glycosidase (β-Gly;Asβ-Gly;ASAC_1390)$3.2.1.21 3.2.1.23 3.2.1.-$Acidilobus saccharovorans 345-15$Archaea$GH1',
 'MAVTFPKDFLFGWSQAGFQSEMGTPGSEDPNSDWYAWVHDRENIAAGLVSGDFPENGPGYWGNYRKFHDAAQAMGLTAARIGVEWSRIFPRPTFDVKVDAEVKGDDVLSVYVSEGALEQLDKMANRDAINHYREMFSDLRSRGITFILNLYHWPLPLWLHDPIAIRRGNLSAPSGWLDVRTVIEFAKFSAYVAWKLDDLVYMYSTMNEPNVVWGLGYAAVKSGFPPGYLCLECAGRAMKNLVQAHARAYDAVKAITKKPVGVIYANSDFTPLTDADREAAERAKFDNRWAFFDAVVRGQLGGSTRDDLKGRLDWIGVNYYTRQVVRARGSGYEIVPGYGHGCEPNGVSPAGRPCSDFGWEFYPEGLYNVLKEYWDRYHLPLLVTENGIADEGDYQRPYYLVSHVYQVHRALQDGVNVIGYLHWSLADNYEWASGFSKRFGLLMVDYSTKRLHWRPSAFIYREIAKSRAITDEIEHLNSVPPLRGLSPGHR',
 '>ABW01253.1$Cmaq_0408$3.2.1.21$Caldivirga maquilingensis IC-167$Archaea$GH1',
 'MDISFPKSFRFGWSQAGFQSEMGTPGSEDPNTDWYVWVHDPENIASGLVSGDLPEHGPGYWGLYRMFHDNAVKMGLDIARINVEWSRIFPKPMPDPPQGNVEVKGNDVLAVHVDENDLKRLDEAANQEAVRHYREIFSDLKARGIHFILNFYHWPLPLWVHDPIRVRKGDLSGPTGWLDVKTVINFARFAAYTAWKFDDLADEYSTMNEPNVVHSNGYMWVKSGFPPSYLNFELSRRVMVNLIQAHARAYDAVKAISKKPIGIIYANSSFTPLTDKDAKA

In [4]:
import pandas as pd
import os
import numpy as np

In [5]:
with open('all_GH1.fasta','w',encoding='utf-8') as otp:
    for ii in cazy_acc_seq:
        otp.write(ii+'\n')

In [15]:
#perform CD-hit step
cmd='cd-hit -i EC4_GH8.fasta -o EC4_GH8_50.fasta -c 0.50'
os.system(cmd)

1

In [81]:
acc,seq=[],[]
with open('EC4_GH8_50.fasta','r',encoding='utf-8') as fasta_file:
    for ii in fasta_file:
        if ii.startswith('>'):
            acc.append(ii.rstrip())
        else:
            seq.append(ii.rstrip()) 

In [108]:
acc_seq={}
for i in range(len(acc)):
    accc=acc[i].split('$')
    acc_no=accc[0]
    acc_seq[acc_no[1:]] = seq[i]
acc_seq

{'ABN51508.1': 'MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPTSIADNQSEVTAMLKAEWEDWKSKRITSNGAGGYKRVQRDASTNYDTVSEGMGYGLLLAVCFNEQALFDDLYRYVKSHFNGNGLMHWHIDANNNVTSHDGGDGAATDADEDIALALIFADKLWGSSGAINYGQEARTLINNLYNHCVEHGSYVLKPGDRWGGSSVTNPSYFAPAWYKVYAQYTGDTRWNQVADKCYQIVEEVKKYNNGTGLVPDWCTASGTPASGQSYDYKYDATRYGWRTAVDYSWFGDQRAKANCDMLTKFFARDGAKGIVDGYTIQGSKISNNHNASFIGPVAAASMTGYDLNFAKELYRETVAVKDSEYYGYYGNSLRLLTLLYITGNFPNPLSDLSGQPTPPSNPTPSLPPQVVYGDVNGDGNVNSTDLTMLKRYLLKSVTNINREAADVNRDGAINSSDMTILKRYLIKSIPHLPY',
 'AAA22409.1': 'MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSIITNSSQVSAVAKEMKPFPQQVNYSGILKPNHVSQESLNNAVKNYYNDWKKKYLKNDLSSLPGGYYVKGEITGNPDGFRPLGTSEGQGYGMIITVLMAGHDSNAQTIYDGLFKTARAFKSSINPNLMGWVVADDKKAQGHFDSATDGDLDIAYSLLLAHKQWGSSGKINYLKEAQNMITKGIKASNVTKNNGLNLGDWGDKSTFDTRPSDWMMSHLRAFYEFTGDKTWLNVIDNLYNTYTNFTNKYSPKTGLISDFVVKNPPQPAPKDFLDESKYTDSYYYNASRVPLRIVMDYAMYGEKRGKVISDKVATWIKSKTKGNPSKIVDGYKLDGTNIGDYPTAVYVSPFIAAGTTNSKNQEWVNSGWDWMKNKKESYFSDSYNLLTMLFLTGNWWKPIPDEKKIQSPINLEVQSELKEQD',
 'AAG49556.1': 'MLKPMW

In [109]:
acc_vs_seq=pd.DataFrame.from_dict(acc_seq, orient = 'index')
acc_vs_seq.set_axis(['Sequence'], axis=1 , inplace=True)
acc_vs_seq

,Sequence
ABN51508.1,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...
AAA22409.1,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...
AAG49556.1,MLKPMWRGWILMLMVWFSVSATAATGWETYKSRFMTADGRIQDTGN...
ACU11859.1,MVALVLAAANARAACSWPAWEQFKQDYISDGGRVIDPSDARKISTS...
BAM44856.1,MKALRWVVMALVLAAMNVRAACTWPAWEHFKRAYISDGGRVIDPSD...
CAB89803.1,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...
AAC76556.1,MNVLRSGIVTMLLLAAFSVQAACTWPAWEQFKKDYISQEGRVIDPS...
ACX73853.1,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...
ABU45499.1,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...
ACN29537.1,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...


In [110]:
import numpy as np
acc_vs_seq['Length']=np.nan
acc_vs_seq

,Sequence,Length
ABN51508.1,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...,NaN
AAA22409.1,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,NaN
AAG49556.1,MLKPMWRGWILMLMVWFSVSATAATGWETYKSRFMTADGRIQDTGN...,NaN
ACU11859.1,MVALVLAAANARAACSWPAWEQFKQDYISDGGRVIDPSDARKISTS...,NaN
BAM44856.1,MKALRWVVMALVLAAMNVRAACTWPAWEHFKRAYISDGGRVIDPSD...,NaN
CAB89803.1,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...,NaN
AAC76556.1,MNVLRSGIVTMLLLAAFSVQAACTWPAWEQFKKDYISQEGRVIDPS...,NaN
ACX73853.1,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...,NaN
ABU45499.1,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...,NaN
ACN29537.1,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...,NaN


In [115]:
for i in range(len(acc)): 
    acc_vs_seq.iloc[i,1]=int(len(acc_vs_seq.iloc[i,0]))


In [116]:
acc_vs_seq

,Sequence,Length
ABN51508.1,MKNVKKRVGVVLLILAVLGVYMLAMPANTVSAAGVPFNTKYPYGPT...,477.0
AAA22409.1,MVEKRKIFTVLCACGIGFTSYTSCISAAAIDNDTLINNGHKINSSI...,463.0
AAG49556.1,MLKPMWRGWILMLMVWFSVSATAATGWETYKSRFMTADGRIQDTGN...,332.0
ACU11859.1,MVALVLAAANARAACSWPAWEQFKQDYISDGGRVIDPSDARKISTS...,360.0
BAM44856.1,MKALRWVVMALVLAAMNVRAACTWPAWEHFKRAYISDGGRVIDPSD...,367.0
CAB89803.1,MMAMFRAGIILLALVLGLSQASAAEGWGTFKTRFLTSEGRIQDTGN...,333.0
AAC76556.1,MNVLRSGIVTMLLLAAFSVQAACTWPAWEQFKKDYISQEGRVIDPS...,368.0
ACX73853.1,MLQRYPTRDLFVEAGYGPNFADQLIQNAYSKLFEGDPIDERIYFEA...,371.0
ABU45499.1,MIFDMLSNYLLGWMGEIFFLKGIHMKNLAKVMFGVAAVAAVTASAG...,752.0
ACN29537.1,MNGKSVKAAVIYLLGGLLVLLGLASMPASANANFAVGDPGWEAYKK...,350.0


In [117]:
filename='EC4_GH8_50.csv'
acc_vs_seq.to_csv(filename)